In [1]:
from kgtk.functions import kgtk, kypher
import os
import gzip
from kgtk.io.kgtkreader import KgtkReader
import json
import sys
from pathlib import Path

In [36]:
os.environ['STORE'] = '/Volumes/saggu-ssd/wikidata.sqlite3.db'
os.environ['sitelinks'] = '/Volumes/saggu-ssd/sitelinks.tsv.gz'

In [37]:
!kgtk --debug query --gc $STORE --show-cache

Graph Cache:
DB file: /Volumes/saggu-ssd/wikidata.sqlite3.db
  size:  294.86 GB   	free:  0 Bytes   	modified:  2022-06-08 18:27:22

KGTK File Information:
alias:
  size:  179.00 MB   	modified:  2022-06-02 19:10:14   	graph:  graph_3
claims:
  size:  16.26 GB   	modified:  2022-06-08 12:55:05   	graph:  graph_5
classvizedge:
  size:  595.03 MB   	modified:  2022-05-20 17:24:07   	graph:  graph_8
classviznode:
  size:  502.27 MB   	modified:  2022-05-20 17:24:22   	graph:  graph_9
datatypes:
  size:  54.71 KB   	modified:  2022-06-02 19:12:17   	graph:  graph_6
description:
  size:  417.24 MB   	modified:  2022-06-02 19:12:02   	graph:  graph_4
l_d_pgr_ud:
  size:  1.33 GB   	modified:  2022-06-08 14:11:48   	graph:  graph_10
label:
  size:  662.80 MB   	modified:  2022-06-02 19:09:20   	graph:  graph_1
pagerank_undirected:
  size:  769.02 MB   	modified:  2022-05-20 17:24:43   	graph:  graph_2
qualifiers:
  size:  4.28 GB   	modified:  2022-06-02 19:47:11   	graph:  graph_7

Graph Tab

In [41]:
kypher(""" --debug
            -i claims -i qualifiers -i label --gc $STORE
            --match 'claims: (n1:Q12073038)-[l:P26]->(n2),
                     label: (n1:Q12073038)-[]->(n1_label),
                     label: (n2)-[]->(n2_label)'
            --opt 'qualifiers: (l)-[:P580]->(start_time)'
            --return 'n1 as node1, 
                      kgtk_lqstring_text(n1_label) as node1_label, 
                      "married to" as label, 
                      n2 as node2, 
                      kgtk_lqstring_text(n2_label) as node2_label, 
                      substr(kgtk_date_date(start_time), 2) as start_time'
            --order-by 'start_time desc'
            
""")

[2022-06-09 16:39:28 query]: SQL Translation:
---------------------------------------------
  SELECT graph_5_c1."node1" "_aLias.node1", kgtk_lqstring_text(graph_1_c2."node2") "_aLias.node1_label", ? "_aLias.label", graph_5_c1."node2" "_aLias.node2", kgtk_lqstring_text(graph_1_c3."node2") "_aLias.node2_label", substr(kgtk_date_date(graph_7_c4."node2"), ?) "_aLias.start_time"
     FROM graph_1 AS graph_1_c2
     INNER JOIN graph_1 AS graph_1_c3, graph_5 AS graph_5_c1
     ON graph_5_c1."node1" = graph_1_c2."node1"
        AND graph_5_c1."node2" = graph_1_c3."node1"
        AND graph_1_c2."node1" = ?
        AND graph_5_c1."label" = ?
        AND graph_5_c1."node1" = ?
     LEFT JOIN graph_7 AS graph_7_c4
     ON graph_5_c1."id" = graph_7_c4."node1"
        AND graph_7_c4."label" = ?
     ORDER BY "_aLias.start_time" DESC
  PARAS: ['married to', 2, 'Q12073038', 'P26', 'Q12073038', 'P580']
---------------------------------------------



,node1,node1_label,label,node2,node2_label,start_time
0,Q12073038,Jennifer Winget,married to,Q3521637,Karan Singh Grover,2012-01-01


In [40]:
kypher(""" --debug
            -i claims -i qualifiers --gc $STORE
            --match 'claims: (n1:Q12073038)-[l:P26]->(n2)'
            --opt 'qualifiers: (l)-[:P580]->(start_time)'
            
""")

[2022-06-09 16:39:21 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_5 AS graph_5_c1
     LEFT JOIN graph_7 AS graph_7_c2
     ON graph_5_c1."id" = graph_7_c2."node1"
        AND graph_7_c2."label" = ?
     WHERE graph_5_c1."label" = ?
        AND graph_5_c1."node1" = ?
  PARAS: ['P580', 'P26', 'Q12073038']
---------------------------------------------



,node1,label,node2,id,lang,rank,node2;wikidatatype,node1.1,label.1,node2.1,id.1,lang.1,rank.1,node2;wikidatatype.1
0,Q12073038,P26,Q3521637,Q12073038-P26-Q3521637-4583fb83-0,NaN,normal,wikibase-item,Q12073038-P26-Q3521637-4583fb83-0,P580,^2012-01-01T00:00:00Z/9,Q12073038-P26-Q3521637-4583fb83-0-P580-7de3d9-0,NaN,NaN,time


In [39]:
kypher(""" --debug
            -i claims --gc $STORE
            --match 'claims: (n1:Q12073038)-[l:P26]->(n2)'
            
""")

[2022-06-09 16:39:15 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_5 AS graph_5_c1
     WHERE graph_5_c1."label" = ?
        AND graph_5_c1."node1" = ?
  PARAS: ['P26', 'Q12073038']
---------------------------------------------



,node1,label,node2,id,lang,rank,node2;wikidatatype
0,Q12073038,P26,Q3521637,Q12073038-P26-Q3521637-4583fb83-0,NaN,normal,wikibase-item


In [38]:
kypher(""" --debug
            -i claims --gc $STORE
            --match 'claims: (n1:Q12073038)-[]->(n2)'
            
""")

[2022-06-09 16:38:51 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_5 AS graph_5_c1
     WHERE graph_5_c1."node1" = ?
  PARAS: ['Q12073038']
---------------------------------------------



,node1,label,node2,id,lang,rank,node2;wikidatatype
0,Q12073038,Pshort_abstract,Jennifer Winget (born 30 May 1985) is an India...,NaN,NaN,NaN,NaN
1,Q12073038,P103,Q1568,Q12073038-P103-Q1568-4e9b6967-0,NaN,normal,wikibase-item
2,Q12073038,P106,Q10798782,Q12073038-P106-Q10798782-ae153a11-0,NaN,normal,wikibase-item
3,Q12073038,P106,Q33999,Q12073038-P106-Q33999-b10ca5ed-0,NaN,normal,wikibase-item
4,Q12073038,P1412,Q1568,Q12073038-P1412-Q1568-406d094f-0,NaN,normal,wikibase-item
5,Q12073038,P18,Jennifer Winget at the 25th SOL Lions Gold Awa...,Q12073038-P18-803526-4dfadde7-0,NaN,normal,commonsMedia
6,Q12073038,P19,Q1708373,Q12073038-P19-Q1708373-fd52a17b-0,NaN,normal,wikibase-item
7,Q12073038,P2002,jenwinget,Q12073038-P2002-d44505-59ea0143-0,NaN,normal,external-id
8,Q12073038,P2003,jenniferwinget1,Q12073038-P2003-55843e-c732739a-0,NaN,normal,external-id
9,Q12073038,P2013,jenniferwinget1,Q12073038-P2013-55843e-4b9b81e6-0,NaN,normal,external-id


In [42]:
kypher(""" --debug
            -i claims -i qualifiers -i label --gc $STORE
            --match 'claims: (n1)-[l:P26]->(n2),
                     label: (n1)-[]->(n1_label),
                     label: (n2)-[]->(n2_label)'
            --opt 'qualifiers: (l)-[:P580]->(start_time)'
            --return 'n1 as node1, 
                      kgtk_lqstring_text(n1_label) as node1_label, 
                      "married to" as label, 
                      n2 as node2, 
                      kgtk_lqstring_text(n2_label) as node2_label, 
                      substr(kgtk_date_date(start_time), 2) as start_time'
            --order-by 'start_time desc'
            -o /tmp/marriages_in_wikidata.tsv.gz
""")

[2022-06-09 16:39:38 query]: SQL Translation:
---------------------------------------------
  SELECT graph_5_c1."node1" "_aLias.node1", kgtk_lqstring_text(graph_1_c2."node2") "_aLias.node1_label", ? "_aLias.label", graph_5_c1."node2" "_aLias.node2", kgtk_lqstring_text(graph_1_c3."node2") "_aLias.node2_label", substr(kgtk_date_date(graph_7_c4."node2"), ?) "_aLias.start_time"
     FROM graph_1 AS graph_1_c2
     INNER JOIN graph_1 AS graph_1_c3, graph_5 AS graph_5_c1
     ON graph_5_c1."node1" = graph_1_c2."node1"
        AND graph_5_c1."node2" = graph_1_c3."node1"
        AND graph_5_c1."label" = ?
     LEFT JOIN graph_7 AS graph_7_c4
     ON graph_5_c1."id" = graph_7_c4."node1"
        AND graph_7_c4."label" = ?
     ORDER BY "_aLias.start_time" DESC
  PARAS: ['married to', 2, 'P26', 'P580']
---------------------------------------------



In [43]:
!gzcat /tmp/marriages_in_wikidata.tsv.gz | head

node1	node1_label	label	node2	node2_label	start_time
Q105432158	Imra Ardeen	married to	Q96108374	Mon-El	3100-01-01
Q96108374	Mon-El	married to	Q105432158	Imra Ardeen	3100-01-01
Q1063412	Ranbir Kapoor	married to	Q4725343	Alia Bhatt	2022-04-14
Q4725343	Alia Bhatt	married to	Q1063412	Ranbir Kapoor	2022-04-14
Q272031	Jack White	married to	Q66049378	Olivia Jean	2022-04-08
Q66049378	Olivia Jean	married to	Q272031	Jack White	2022-04-08
Q30177812	Kristen Bicknell	married to	Q54626146	Alex Foxen	2022-04-04
Q54626146	Alex Foxen	married to	Q30177812	Kristen Bicknell	2022-04-04
Q220140	Travis Barker	married to	Q575689	Kourtney Kardashian	2022-03-29
gzcat: error writing to output: Broken pipe
gzcat: /tmp/marriages_in_wikidata.tsv.gz: uncompress failed


In [44]:
!gzcat /tmp/marriages_in_wikidata.tsv.gz | wc -l

  711261


In [45]:
def read_sitelinks(sitelinks_file):
    f = gzip.open(sitelinks_file, 'rt')
    sitelinks_d = dict()
    for line in f:
        vals = line.split('\t')
        node = vals[1]
        lang = vals[4]
        url = vals[3]
        prop = vals[2]
        if prop == 'wikipedia_sitelink' and lang == 'en':
            sitelinks_d[node] = url
    return sitelinks_d

In [46]:
sitelinks_d = read_sitelinks(os.environ['sitelinks'])

In [61]:
sitelinks_d['Q12073038']

'http://en.wikipedia.org/wiki/Jennifer_Winget'

In [56]:
def first_name(name):
    if len(name.split(' ')) == 1:
           return name
    return ' '.join(name.split(' ')[:-1])

In [57]:
first_name('Karan Singh Grover')

'Karan Singh'

In [58]:
first_name('Karan')

'Karan'

In [68]:
def create_marriage_questions(married_file, o_answers, o_no_answers):
    kr = KgtkReader.open(Path(married_file),
                          error_file=sys.stderr,
                          who="married"
                          )
    node1_lbl_idx = kr.column_name_map['node1_label']
    node2_lbl_idx = kr.column_name_map['node2_label']
    node1_idx = kr.column_name_map['node1']
    node2_idx = kr.column_name_map['node2']
    st_idx = kr.column_name_map['start_time']
    seen_questions = set()
    questions_with_answers = []
    questions_no_answers = []
    for row in kr:
        node1 = row[node1_idx]
        node2 = row[node2_idx]
        node1_l = row[node1_lbl_idx]
        node2_l = row[node2_lbl_idx]
        answer = row[st_idx].strip()
        if node1 in sitelinks_d and node2 in sitelinks_d:
            question_1 = f"when did {node1_l} marry {first_name(node2_l)}?"
            question_2 = f"when did {node2_l} marry {first_name(node1_l)}?"
            if question_1 not in seen_questions and question_2 not in seen_questions:
                if answer != '':
                    questions_with_answers.append({'q': question_1, 
                                      'a': answer, 
                                      'n1': node1, 
                                      'n2': node2, 
                                      'n1_label': node1_l, 
                                      'n2_label': node2_l,
                                      'url': sitelinks_d[node1]})
                    questions_with_answers.append({'q': question_2, 
                                      'a': answer, 
                                      'n1': node2, 
                                      'n2': node1, 
                                      'n1_label': node2_l, 
                                      'n2_label': node1_l,
                                      'url': sitelinks_d[node2]})
                else:
                    questions_no_answers.append({'q': question_1, 
                                      'a': answer, 
                                      'n1': node1, 
                                      'n2': node2, 
                                      'n1_label': node1_l, 
                                      'n2_label': node2_l,
                                      'url': sitelinks_d[node1]})
                    questions_no_answers.append({'q': question_2, 
                                      'a': answer, 
                                      'n1': node2, 
                                      'n2': node1, 
                                      'n1_label': node2_l, 
                                      'n2_label': node1_l,
                                      'url': sitelinks_d[node2]})

                seen_questions.add(question_1)
                seen_questions.add(question_2)
        elif node1 in sitelinks_d:
            question = f"when did {node1_l} marry {first_name(node2_l)}?"     
            if question not in seen_questions:
                if answer != '':
                    questions_with_answers.append({'q': question, 
                                      'a': answer, 
                                      'n1': node1, 
                                      'n2': node2, 
                                      'n1_label': node1_l, 
                                      'n2_label': node2_l,
                                      'url': sitelinks_d[node1]})

                else:
                    questions_no_answers.append({'q': question, 
                                      'a': answer, 
                                      'n1': node1, 
                                      'n2': node2, 
                                      'n1_label': node1_l, 
                                      'n2_label': node2_l,
                                      'url': sitelinks_d[node1]})
            seen_questions.add(question)
        elif node2 in sitelinks_d:
            question = f"when did {node2_l} marry {first_name(node1_l)}?"     
            if question not in seen_questions:
                if answer != '':
                    questions_with_answers.append({'q': question, 
                                      'a': answer, 
                                      'n1': node2, 
                                      'n2': node1, 
                                      'n1_label': node2_l, 
                                      'n2_label': node1_l,
                                      'url': sitelinks_d[node2]})

                else:
                    questions_no_answers.append({'q': question, 
                                      'a': answer, 
                                      'n1': node2, 
                                      'n2': node1, 
                                      'n1_label': node2_l, 
                                      'n2_label': node1_l,
                                      'url': sitelinks_d[node2]})
            seen_questions.add(question)
    open(o_answers, 'w').write(json.dumps(questions_with_answers))
    open(o_no_answers, 'w').write(json.dumps(questions_no_answers))
    print(len(questions_with_answers))
    print(len(questions_no_answers))
    kr.close()

In [69]:
create_marriage_questions('/tmp/marriages_in_wikidata.tsv.gz',
                          '/tmp/marriages_in_wikidata.json',
                         '/tmp/marriages_in_wikidata_no_answers.json')

68159
69207


In [70]:
mq = json.load(open('/tmp/marriages_in_wikidata.json'))

In [71]:
for h in mq:
    if h['n1'] == 'Q12073038' or h['n2'] == 'Q12073038':
        print(h)

{'q': 'when did Jennifer Winget marry Karan Singh?', 'a': '2012-01-01', 'n1': 'Q12073038', 'n2': 'Q3521637', 'n1_label': 'Jennifer Winget', 'n2_label': 'Karan Singh Grover', 'url': 'http://en.wikipedia.org/wiki/Jennifer_Winget'}
{'q': 'when did Karan Singh Grover marry Jennifer?', 'a': '2012-01-01', 'n1': 'Q3521637', 'n2': 'Q12073038', 'n1_label': 'Karan Singh Grover', 'n2_label': 'Jennifer Winget', 'url': 'http://en.wikipedia.org/wiki/Karan_Singh_Grover'}
